# Geospatial data analysis of time series
Quinn Brencher

In [ ]:
# Import required packages
%matplotlib inline
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio import plot, mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import matplotlib.pyplot as plt
import h5py
import xarray as xr
import datetime as dt

## Load in and rebuild time series data

In [ ]:
os.chdir('/tmp')

if not os.path.exists('/tmp/polygons'):
        os.makedirs('/tmp/polygons')

os.chdir('/tmp/mintpy')

Place json with areas of interest or stable reference areas in polyons folder

In [ ]:
# open time series data 
disp_fn = '/tmp/mintpy/timeseries_ERA5_ramp_demErr.h5'
disp_df = xr.open_dataset(disp_fn, engine='h5netcdf', phony_dims='sort', decode_coords="all")

In [ ]:
# examine structure
# MintPy outputs H5 files and the coordinates don't survive being loaded into xarray
disp_df 

In [ ]:
# rename variables
disp_df = disp_df.rename({'date': 'dstring',
                          'timeseries': 'displacement'})

In [ ]:
# rename dims
disp_df = disp_df.rename_dims({'phony_dim_0':'date',
                               'phony_dim_1':'latitude',
                               'phony_dim_2':'longitude'})

In [ ]:
# Already in wgs84, just has no coordinates
disp_df.isel(date=7).displacement.plot.imshow()

In [ ]:
# use the 'date' data variable to create date list
date_list = []
for i in disp_df['dstring']:
    date_list.append(dt.datetime.strptime(str(i)[40:48], '%Y%m%d'))

In [ ]:
# function to rewrite coordinates from metadata
def coord_range(df, coord):
    if coord == 'lat' or coord == 'latitude':
        latrange = np.linspace(float(df.attrs['Y_FIRST']),((float(df.attrs['Y_STEP'])*float(df.attrs['LENGTH']))+float(df.attrs['Y_FIRST'])),int(df.attrs['LENGTH']))
        return latrange
    elif coord == 'lon' or coord == 'longitude':
        lonrange = np.linspace(float(df.attrs['X_FIRST']),((float(df.attrs['X_STEP'])*float(df.attrs['WIDTH']))+float(df.attrs['X_FIRST'])),int(df.attrs['WIDTH']))
        return lonrange

In [ ]:
# rebuild coordinates
disp_df = disp_df.assign_coords({'date': ('date', np.array(date_list)),
                                 'latitude': ('latitude', coord_range(disp_df, 'lat')),
                                 'longitude': ('longitude', coord_range(disp_df, 'lon'))})

In [ ]:
# Examine object
disp_df

In [ ]:
# set correct CRS
disp_df = disp_df.rio.write_crs(32645)
disp_df.rio.crs

In [ ]:
# Sanity check
disp_df.isel(date=7).displacement.plot.imshow()

In [ ]:
# Probably not much use reprojecting to utm in this case. The site is small and within a degree of the equator. 

## Load in average velocity

In [ ]:
# load in average velocity
vel_fn = '/tmp/mintpy/velocity.h5'
vel_df = xr.open_dataset(vel_fn, engine='h5netcdf', phony_dims='sort', decode_coords="all")

In [ ]:
#examine structure
vel_df

In [ ]:
# rebuild dataset
vel_df = vel_df.rename_dims({'phony_dim_0':'latitude',
                             'phony_dim_1':'longitude'})

vel_df = vel_df.assign_coords({'latitude': ('latitude', coord_range(vel_df, 'lat')), 
                               'longitude': ('longitude', coord_range(vel_df, 'lon'))})

vel_df = vel_df.rio.write_crs(4326)
vel_df

## Load in temporal coherence

In [ ]:
# load in average velocity
coh_fn = '/tmp/mintpy/temporalCoherence.h5'
coh_df = xr.open_dataset(coh_fn, engine='h5netcdf', phony_dims='sort', decode_coords="all")

In [ ]:
#examine structure
coh_df

In [ ]:
# rebuild dataset
coh_df = coh_df.rename_dims({'phony_dim_0':'latitude',
                             'phony_dim_1':'longitude'})

coh_df = coh_df.assign_coords({'latitude': ('latitude', coord_range(coh_df, 'lat')), 
                               'longitude': ('longitude', coord_range(coh_df, 'lon'))})

coh_df

In [ ]:
# convert to tif for download
coh_df.rio.to_raster('2017_current_coh.tif')

## Load in polygons

In [ ]:
# load json of aoi 
aoi_fn = '~/Friendly-InSAR-time-series/moraine_wlakes.geojson'
aoi_gdf = gpd.read_file(aoi_fn)

In [ ]:
# load json of stable reference areas
ref_fn = '~/Friendly-InSAR-time-series/local_ref_polygon.geojson'
ref_gdf = gpd.read_file(ref_fn)

In [ ]:
# Sanity check
f, ax = plt.subplots(figsize=(10,8))
vel_df.velocity.plot(ax=ax, cbar_kwargs={'label':'velocity (m/yr)'})
aoi_gdf.plot(ax=ax, facecolor='none', edgecolor='r')
ref_gdf.plot(ax=ax,  facecolor='none', edgecolor='gray')
ax.set_title('velocity, reference areas, and area of interest');

## Displacement of aoi and reference areas

In [ ]:
# clip to aoi, clip to reference areas
disp_aoi = disp_df.rio.clip(aoi_gdf.geometry, crs=aoi_gdf.crs, drop=False)
disp_ref = disp_df.rio.clip(ref_gdf.geometry, crs=ref_gdf.crs, drop=False)

In [ ]:
plt.style.use('ggplot')

# Look at displacement distribution in aoi and reference area
f,ax = plt.subplots(figsize=(8,4))
ax.hist(np.ravel(disp_ref.displacement), bins=200, density=True, alpha=0.7, label='reference pixels')
ax.hist(np.ravel(disp_aoi.displacement), bins=200, density=True, alpha=0.7, label='aoi pixels')
ax.set_title('displacement in aoi pixels and reference pixels')
ax.set_ylabel('normalized count')
ax.set_xlabel('displacement (m/yr)')
ax.set_ylim(0,110)
ax.set_xlim(-0.1,0.25)
ax.legend();

In [ ]:
# look at cumulative displacement in aoi and reference area
f, ax = plt.subplots(figsize=(10,5))
disp_aoi.median(dim=('latitude', 'longitude')).displacement.plot(ax=ax, label='median', c='k')
disp_aoi.max(dim=('latitude', 'longitude')).displacement.plot(ax=ax, label='max', c='c', alpha=0.5)
disp_aoi.min(dim=('latitude', 'longitude')).displacement.plot(ax=ax, label='min', c='r', alpha=0.5)
disp_ref.median(dim=('latitude', 'longitude')).displacement.plot(ax=ax, label='median uncertainty', c='orange')
ax.set_title('Mean cumulative displacement of Imja Lake dam')
ax.set_ylabel('displacement (m)')
ax.set_xlabel('time')
ax.legend();

In [ ]:
f, ax = plt.subplots(figsize=(5.8,2))
disp_aoi.median(dim=('latitude', 'longitude')).displacement.plot(ax=ax, label='dam median', c='lightsalmon')
disp_ref.median(dim=('latitude', 'longitude')).displacement.plot(ax=ax, label='stable area median', c='gray')
disp_aoi.min(dim=('latitude', 'longitude')).displacement.plot(ax=ax, label='dam maximum', c='r', alpha=0.5)
ax.set_title('Mean cumulative displacement of Imja Lake dam')
ax.set_ylabel('displacement (m)')
ax.set_xlabel('time')
ax.legend();

## Mean velocity of aoi and reference areas

In [ ]:
# clip to aoi extent, clip to reference area extent
vel_aoi = vel_df.rio.clip(aoi_gdf.geometry, crs=aoi_gdf.crs, drop=False)
vel_ref = vel_df.rio.clip(ref_gdf.geometry, crs=ref_gdf.crs, drop=False)

In [ ]:
# look at velocity distribution in aoi and reference area
f,ax = plt.subplots(figsize=(8,4))
ax.hist(np.ravel(vel_ref.velocity), bins=50, density=True, alpha=0.7, label='reference pixels')
ax.hist(np.ravel(vel_aoi.velocity), bins=50, density=True, alpha=0.7, label='aoi pixels')
ax.set_title('mean velocity in aoi pixels and reference pixels')
ax.set_ylabel('normalized count')
ax.set_xlabel('velocity (m/yr)')
ax.set_ylim(0,300)
ax.legend();

In [ ]:
# make 1d and remove na for plotting
aoi_filt = np.ravel(vel_aoi.velocity)[~np.isnan(np.ravel(vel_aoi.velocity))]
ref_filt = np.ravel(vel_ref.velocity)[~np.isnan(np.ravel(vel_ref.velocity))]

In [ ]:
## Mean velocity in aoi and reference area
f, ax = plt.subplots(1, 2, figsize=(6,5), sharey=True)
ax[0].boxplot(aoi_filt, widths=0.8)
ax[1].boxplot(ref_filt, widths=0.8)
ax[0].set_title('Mean velocity in aoi')
ax[1].set_title('Mean velocity in reference area')
ax[0].set_ylabel('velocity (m/yr)')

## Change reference points 

In [ ]:
# function to find value of new reference point in each time slice and subtract it from data array
rref_list = []

def change_ref(xarray, lat, lon):
    reref = xarray.copy(deep=True)
    for i in range(xarray.sizes['date']):
        # get value from grid
        rref_list.append(float(xarray.isel(date=i).displacement.sel(longitude=lon, latitude=lat, method="nearest").values))
    reref['displacement'] = disp_df['displacement'] - xr.DataArray(rref_list, dims='date')
    return reref       

In [ ]:
#Sanity check: reference point in highly degforming area
disp_rref = change_ref(disp_df, -0.38, -91.525)

In [ ]:
# Sanity check: plot cumulative displacement at new and old reference points
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
disp_df.isel(date=97)['displacement'].plot.imshow(ax=ax[0])
ax[0].set_title('total displacement with original reference point')

disp_rref.isel(date=97).displacement.plot.imshow(ax=ax[1])
ax[1].set_title('total displacement with new reference point');

## Convert to tif for download

In [ ]:
vel_df.rio.to_raster('des_veloc.tif')

## Interactive displacement

In [ ]:
import holoviews as hv

hv.extension('bokeh')

In [ ]:
%%output holomap='scrubber'
%%opts Image style(cmap='RdBu_r') plot[colorbar=True]
%%opts Image [width=500, height=400]
hv_ds = hv.Dataset(disp_df.displacement)
hv_ds.to(hv.Image, ['longitude', 'latitude'])